<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

**<h1 align=center><font size = 5>Image Classifier & Predictor using ResNet50 & VGG16 pre-trained models, Performance Evaluation and Testing of the models against a dedicated test set</font></h1>**

**<h1 align=center><font size = 5>Owner & Author: [Sandeep Aswathnarayana](https://www.linkedin.com/in/sandeep-a/)</font></h1>**
**<h1 align=center><font size = 5>Advisor: [Dr. Alex Aklson](https://www.linkedin.com/in/aklson/)</font></h1>**  
<h3 align=center><font size = 5>Date: March 24, 2020</font></h3>  

NOTE: Please refer to the "Peer Review Final Assignment" section of this notebook for Image Classifier built using VGG16. Also, Evaluation and comparison of its Performance against ResNet50.  

## Introduction


In this lab, you will learn how to leverage pre-trained models to build image classifiers instead of building a model from scratch.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3> 
    
1. <a href="#item31">Import Libraries and Packages</a>
2. <a href="#item32">Download Data</a>  
3. <a href="#item33">Define Global Constants</a>  
4. <a href="#item34">Construct ImageDataGenerator Instances</a>  
5. <a href="#item35">Compile and Fit Model</a>

</font>
    
</div>

## Import Libraries and Packages

Let's start the lab by importing the libraries that we will be using in this lab.

First, we will import the ImageDataGenerator module since we will be leveraging it to train our model in batches.

In [0]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In this lab, we will be using the Keras library to build an image classifier, so let's download the Keras library.

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Finally, we will be leveraging the ResNet50 model to build our classifier, so let's download it as well.

In [0]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

<a id='item32'></a>

## Download Data

For your convenience, I have placed the data on a server which you can retrieve easily using the **wget** command. So let's run the following line of code to get the data. Given the large size of the image dataset, it might take some time depending on your internet speed.

In [0]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

--2020-03-24 07:51:02--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261482368 (249M) [application/zip]
Saving to: ‘concrete_data_week3.zip’

concrete_data_week3 100%[===================>] 249.37M  20.5MB/s    in 13s     

2020-03-24 07:51:16 (18.5 MB/s) - ‘concrete_data_week3.zip’ saved [261482368/261482368]



And now if you check the left directory pane, you should see the zipped file *concrete_data_week3.zip* appear. So, let's go ahead and unzip the file to access the images. Given the large number of images in the dataset, this might take a couple of minutes, so please be patient, and wait until the code finishes running.

In [0]:
!unzip concrete_data_week3.zip

Streaming output truncated to the last 5000 lines.
  inflating: concrete_data_week3/train/negative/08835.jpg  
  inflating: __MACOSX/concrete_data_week3/train/negative/._08835.jpg  
  inflating: concrete_data_week3/train/negative/08821.jpg  
  inflating: __MACOSX/concrete_data_week3/train/negative/._08821.jpg  
  inflating: concrete_data_week3/train/negative/03974.jpg  
  inflating: __MACOSX/concrete_data_week3/train/negative/._03974.jpg  
  inflating: concrete_data_week3/train/negative/12879.jpg  
  inflating: __MACOSX/concrete_data_week3/train/negative/._12879.jpg  
  inflating: concrete_data_week3/train/negative/09281.jpg  
  inflating: __MACOSX/concrete_data_week3/train/negative/._09281.jpg  
  inflating: concrete_data_week3/train/negative/10908.jpg  
  inflating: __MACOSX/concrete_data_week3/train/negative/._10908.jpg  
  inflating: concrete_data_week3/train/negative/01805.jpg  
  inflating: __MACOSX/concrete_data_week3/train/negative/._01805.jpg  
  inflating: concrete_data_week3

Now, you should see the folder *concrete_data_week3* appear in the left pane. If you open this folder by double-clicking on it, you will find that it contains two folders: *train* and *valid*. And if you explore these folders, you will find that each contains two subfolders: *positive* and *negative*. These are the same folders that we saw in the labs in the previous modules of this course, where *negative* is the negative class and it represents the concrete images with no cracks and *positive* is the positive class and it represents the concrete images with cracks.

**Important Note**: There are thousands and thousands of images in each folder, so please don't attempt to double click on the *negative* and *positive* folders. This may consume all of your memory and you may end up with a **50*** error. So please **DO NOT DO IT**.

<a id='item33'></a>

## Define Global Constants

Here, we will define constants that we will be using throughout the rest of the lab. 

1. We are obviously dealing with two classes, so *num_classes* is 2. 
2. The ResNet50 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
3. We will training and validating the model using batches of 100 images.

In [0]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

<a id='item34'></a>

## Construct ImageDataGenerator Instances

In order to instantiate an ImageDataGenerator instance, we will set the **preprocessing_function** argument to *preprocess_input* which we imported from **keras.applications.resnet50** in order to preprocess our images the same way the images used to train ResNet50 model were processed.

In [0]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

Next, we will use the *flow_from_directory* method to get the training images as follows:

In [0]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


**Your Turn**: Use the *flow_from_directory* method to get the validation images and assign the result to **validation_generator**.

In [0]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')


Found 10001 images belonging to 2 classes.


Double-click __here__ for the solution.
<!-- The correct answer is:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')
-->



<a id='item35'></a>

## Build, Compile and Fit Model

In this section, we will start building our model. We will use the Sequential model class from Keras.

In [0]:
model = Sequential()

Next, we will add the ResNet50 pre-trained model to out model. However, note that we don't want to include the top layer or the output layer of the pre-trained model. We actually want to define our own output layer and train it so that it is optimized for our image dataset. In order to leave out the output layer of the pre-trained model, we will use the argument *include_top* and set it to **False**.

In [0]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))












94658560/94653016 [==============================] - 9s 0us/step


Then, we will define our output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.

In [0]:
model.add(Dense(num_classes, activation='softmax'))

You can access the model's layers using the *layers* attribute of our model object. 

In [0]:
model.layers

You can see that our model is composed of two sets of layers. The first set is the layers pertaining to ResNet50 and the second set is a single layer, which is our Dense layer that we defined above.

You can access the ResNet50 layers by running the following:

In [0]:
model.layers[0].layers

Since the ResNet50 model has already been trained, then we want to tell our model not to bother with training the ResNet part, but to train only our dense output layer. To do that, we run the following.

In [0]:
model.layers[0].trainable = False

And now using the *summary* attribute of the model, we can see how many parameters we will need to optimize in order to train the output layer.

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


Next we compile our model using the **adam** optimizer.

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Before we are able to start the training process, with an ImageDataGenerator, we will need to define how many steps compose an epoch. Typically, that is the number of images divided by the batch size. Therefore, we define our steps per epoch as follows:

In [0]:
steps_per_epoch_training = 30
#= len(train_generator)
steps_per_epoch_validation = 30
#= len(validation_generator)
num_epochs = 2

Finally, we are ready to start training our model. Unlike a conventional deep learning training were data is not streamed from a directory, with an ImageDataGenerator where data is augmented in batches, we use the **fit_generator** method.

In [0]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
30/30 [==============================] - 1712s 57s/step - loss: 0.1961 - acc: 0.9213 - val_loss: 0.3137 - val_acc: 0.8493
Epoch 2/2
30/30 [==============================] - 1737s 58s/step - loss: 0.0349 - acc: 0.9933 - val_loss: 0.3094 - val_acc: 0.8563


Now that the model is trained, you are ready to start using it to classify images.

Since training can take a long time when building deep learning models, it is always a good idea to save your model once the training is complete if you believe you will be using the model again later. You will be using this model in the next module, so go ahead and save your model.

In [0]:
model.save('classifier_resnet_model.h5')

In [0]:
 from keras.models import load_model
 classifier_resnet_model = load_model('classifier_resnet_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


Now, you should see the model file *classifier_resnet_model.h5* apprear in the left directory pane.

#**<center>Peer Review Final Assignment</center>**  

 
##Introduction  
In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!  

Table of Contents  

Download Data  
Part 1  
Part 2  
Part 3  

##Download Data  
Use the wget command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip  

Use the following cells to download the data.  

In [0]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-03-24 10:50:04--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  19.7MB/s    in 14s     

2020-03-24 10:50:19 (17.8 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [0]:
!unzip concrete_data_week4.zip

Streaming output truncated to the last 5000 lines.
  inflating: concrete_data_week4/train/negative/08835.jpg  
  inflating: __MACOSX/concrete_data_week4/train/negative/._08835.jpg  
  inflating: concrete_data_week4/train/negative/08821.jpg  
  inflating: __MACOSX/concrete_data_week4/train/negative/._08821.jpg  
  inflating: concrete_data_week4/train/negative/03974.jpg  
  inflating: __MACOSX/concrete_data_week4/train/negative/._03974.jpg  
  inflating: concrete_data_week4/train/negative/12879.jpg  
  inflating: __MACOSX/concrete_data_week4/train/negative/._12879.jpg  
  inflating: concrete_data_week4/train/negative/09281.jpg  
  inflating: __MACOSX/concrete_data_week4/train/negative/._09281.jpg  
  inflating: concrete_data_week4/train/negative/10908.jpg  
  inflating: __MACOSX/concrete_data_week4/train/negative/._10908.jpg  
  inflating: concrete_data_week4/train/negative/01805.jpg  
  inflating: __MACOSX/concrete_data_week4/train/negative/._01805.jpg  
  inflating: concrete_data_week4

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.  

#Part 1  
In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model VGG16 from keras.applications.  

You will essentially build your classifier as follows:  

1. Import libraries, modules, and packages you will need. Make sure to import the preprocess_input function from keras.applications.vgg16.  
2. Use a batch size of 100 images for both training and validation.  
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.  
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.  
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.  
6. Fit the model on the augmented data using the ImageDataGenerators.  

Use the following cells to create your classifier.  

In [0]:
from keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

In [0]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [0]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [0]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [0]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [0]:
model = Sequential()

In [0]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

58892288/58889256 [==============================] - 6s 0us/step


In [0]:
model.add(Dense(num_classes, activation='softmax'))

In [0]:
model.layers

In [0]:
model.layers[0].layers

In [0]:
model.layers[0].trainable = False

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
steps_per_epoch_training = 20
#= len(train_generator)
steps_per_epoch_validation = 20
#= len(validation_generator)
num_epochs = 2

In [0]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
20/20 [==============================] - 2023s 101s/step - loss: 0.8224 - acc: 0.5890 - val_loss: 0.3963 - val_acc: 0.8355
Epoch 2/2
20/20 [==============================] - 2039s 102s/step - loss: 0.2736 - acc: 0.9135 - val_loss: 0.1618 - val_acc: 0.9655


In [0]:
model.save('vgg16.h5')

In [0]:
from keras.models import load_model
vgg16 = load_model('vgg16.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [0]:
vgg16.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 14,715,714
Non-trainable params: 0
_________________________________________________________________


#Part 2  
In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:  

1. Load your saved model that was built using the ResNet50 model.  
2. Construct an ImageDataGenerator for the test set. 3. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the shuffle parameter and set it to False.  
4. Use the evaluate_generator method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about evaluate_generator here.  
5. Print the performance of the classifier using the VGG16 pre-trained model.  
6. Print the performance of the classifier using the ResNet pre-trained model.  

Use the following cells to evaluate your models.  

In [0]:
from keras.models import load_model
classifier_resnet_model = load_model('classifier_resnet_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [0]:
classifier_resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 23,538,690
Non-trainable params: 53,120
_________________________________________________________________


In [0]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(224, 224),
    batch_size=100,
    class_mode='categorical',
    shuffle=False)

Found 500 images belonging to 2 classes.


In [0]:
len(test_generator)

5

In [0]:
resnet50_score = classifier_resnet_model.evaluate_generator(test_generator,steps=5,verbose=1)

5/5 [==============================] - 156s 31s/step


In [0]:
resnet50_score

[0.3082904428243637, 0.8480000019073486]

In [0]:
classifier_resnet_model.metrics_names

['loss', 'acc']

In [0]:
print("Accuracy for ResNet50 is %.2f%%" % (resnet50_score[1]*100))

Accuracy for ResNet50 is 84.80%


In [0]:
vgg16_score = vgg16.evaluate_generator(test_generator,steps=5,verbose=1)

5/5 [==============================] - 254s 51s/step


In [0]:
print("Accuracy for VGG16 is %.2f%%" % (vgg16_score[1]*100))

Accuracy for VGG16 is 96.60%


#Part 3  
In this model, you will predict whether the images in the test data are images of cracked concrete or not.  You will do the following:  

1. Use the predict_generator method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the predict_generator method here.  
2. Report the class predictions of the first five images in the test set. You should print something list this:  
Positive  
Negative  
Positive  
Positive  
Negative  

Use the following cells to make your predictions.  

In [0]:
resnet50_predict = classifier_resnet_model.predict_generator(generator=test_generator,steps=5,verbose=1)

5/5 [==============================] - 149s 30s/step


In [0]:
resnet50_predict

array([[9.99268711e-01, 7.31322507e-04],
       [1.95862763e-02, 9.80413735e-01],
       [9.95968342e-01, 4.03161580e-03],
       [9.99459803e-01, 5.40250214e-04],
       [1.45197228e-01, 8.54802787e-01],
       [6.48851693e-01, 3.51148367e-01],
       [3.33826207e-02, 9.66617346e-01],
       [9.92382348e-01, 7.61764776e-03],
       [9.84561622e-01, 1.54383536e-02],
       [9.16177273e-01, 8.38227421e-02],
       [3.54006708e-01, 6.45993292e-01],
       [9.80942607e-01, 1.90574657e-02],
       [9.94114101e-01, 5.88596379e-03],
       [9.65247095e-01, 3.47529463e-02],
       [2.76713610e-01, 7.23286390e-01],
       [9.97580290e-01, 2.41963123e-03],
       [3.50184813e-02, 9.64981437e-01],
       [4.80161399e-01, 5.19838631e-01],
       [2.16559812e-01, 7.83440173e-01],
       [9.70538855e-01, 2.94611808e-02],
       [3.39221716e-01, 6.60778284e-01],
       [3.35767508e-01, 6.64232492e-01],
       [9.93711948e-01, 6.28813263e-03],
       [1.26136363e-01, 8.73863578e-01],
       [4.483175

In [0]:
len(resnet50_predict)

500

In [0]:
resnet50_predict.round(2)

array([[1.  , 0.  ],
       [0.02, 0.98],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [0.15, 0.85],
       [0.65, 0.35],
       [0.03, 0.97],
       [0.99, 0.01],
       [0.98, 0.02],
       [0.92, 0.08],
       [0.35, 0.65],
       [0.98, 0.02],
       [0.99, 0.01],
       [0.97, 0.03],
       [0.28, 0.72],
       [1.  , 0.  ],
       [0.04, 0.96],
       [0.48, 0.52],
       [0.22, 0.78],
       [0.97, 0.03],
       [0.34, 0.66],
       [0.34, 0.66],
       [0.99, 0.01],
       [0.13, 0.87],
       [0.45, 0.55],
       [0.99, 0.01],
       [0.96, 0.04],
       [0.27, 0.73],
       [1.  , 0.  ],
       [0.1 , 0.9 ],
       [0.26, 0.74],
       [0.1 , 0.9 ],
       [0.05, 0.95],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [0.  , 1.  ],
       [0.99, 0.01],
       [1.  , 0.  ],
       [0.23, 0.77],
       [1.  , 0.  ],
       [0.13, 0.87],
       [0.99, 0.01],
       [1.  , 0.  ],
       [0.36, 0.64],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [0.89, 0.11],
       [0.18,

In [0]:
#Slice out the second column
resnet50_predict[:,1].round(0)

array([0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0.,
       0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1.

In [0]:
for row in resnet50_predict[0:5]:
    if row[[0]] >= row[[1]]:
        print('Negative')
    else:
        print('Positive')

Negative
Positive
Negative
Negative
Positive


In [0]:
vgg16_predict = vgg16.predict_generator(generator=test_generator,steps=5,verbose=1)

5/5 [==============================] - 254s 51s/step


In [0]:
vgg16_predict

array([[1.28559396e-02, 9.87144053e-01],
       [4.24668156e-02, 9.57533121e-01],
       [2.83428710e-02, 9.71657097e-01],
       [1.08301058e-01, 8.91698956e-01],
       [1.15942312e-02, 9.88405824e-01],
       [1.93105400e-01, 8.06894600e-01],
       [9.24527287e-01, 7.54726827e-02],
       [9.05437112e-01, 9.45628509e-02],
       [8.51835430e-01, 1.48164496e-01],
       [1.65869165e-02, 9.83413100e-01],
       [7.72734731e-02, 9.22726512e-01],
       [8.41205001e-01, 1.58794999e-01],
       [8.76139641e-01, 1.23860277e-01],
       [4.19753138e-03, 9.95802462e-01],
       [5.02534471e-02, 9.49746549e-01],
       [5.46203792e-01, 4.53796208e-01],
       [6.93719864e-01, 3.06280106e-01],
       [3.00684333e-01, 6.99315667e-01],
       [1.63824460e-03, 9.98361766e-01],
       [1.96764693e-02, 9.80323553e-01],
       [6.83349550e-01, 3.16650450e-01],
       [8.50035250e-01, 1.49964735e-01],
       [5.18694893e-03, 9.94813085e-01],
       [9.60236182e-04, 9.99039710e-01],
       [6.786917

In [0]:
len(vgg16_predict)

500

In [0]:
vgg16_predict.round(2)

array([[0.01, 0.99],
       [0.04, 0.96],
       [0.03, 0.97],
       [0.11, 0.89],
       [0.01, 0.99],
       [0.19, 0.81],
       [0.92, 0.08],
       [0.91, 0.09],
       [0.85, 0.15],
       [0.02, 0.98],
       [0.08, 0.92],
       [0.84, 0.16],
       [0.88, 0.12],
       [0.  , 1.  ],
       [0.05, 0.95],
       [0.55, 0.45],
       [0.69, 0.31],
       [0.3 , 0.7 ],
       [0.  , 1.  ],
       [0.02, 0.98],
       [0.68, 0.32],
       [0.85, 0.15],
       [0.01, 0.99],
       [0.  , 1.  ],
       [0.07, 0.93],
       [0.02, 0.98],
       [0.  , 1.  ],
       [0.66, 0.34],
       [0.06, 0.94],
       [0.  , 1.  ],
       [0.03, 0.97],
       [0.87, 0.13],
       [0.15, 0.85],
       [0.  , 1.  ],
       [0.93, 0.07],
       [0.8 , 0.2 ],
       [0.01, 0.99],
       [0.01, 0.99],
       [0.94, 0.06],
       [0.75, 0.25],
       [0.19, 0.81],
       [0.68, 0.32],
       [0.1 , 0.9 ],
       [0.01, 0.99],
       [0.91, 0.09],
       [0.03, 0.97],
       [0.  , 1.  ],
       [0.83,

In [0]:
#Slice out the second column to get all 500 predicted results
vgg16_predict[:,1].round(0)

array([1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
       0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1.,
       0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1.,
       1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1.,
       1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 0., 1., 0., 0.

In [0]:
for row in vgg16_predict[0:5]:
    if row[[0]] >= row[[1]]:
        print('Negative')
    else:
        print('Positive')

Positive
Positive
Positive
Positive
Positive


In [0]:
filenames = test_generator.filenames
filenames

['negative/19751.jpg',
 'negative/19752.jpg',
 'negative/19753.jpg',
 'negative/19754.jpg',
 'negative/19755.jpg',
 'negative/19756.jpg',
 'negative/19757.jpg',
 'negative/19758.jpg',
 'negative/19759.jpg',
 'negative/19760.jpg',
 'negative/19761.jpg',
 'negative/19762.jpg',
 'negative/19763.jpg',
 'negative/19764.jpg',
 'negative/19765.jpg',
 'negative/19766.jpg',
 'negative/19767.jpg',
 'negative/19768.jpg',
 'negative/19769.jpg',
 'negative/19770.jpg',
 'negative/19771.jpg',
 'negative/19772.jpg',
 'negative/19773.jpg',
 'negative/19774.jpg',
 'negative/19775.jpg',
 'negative/19776.jpg',
 'negative/19777.jpg',
 'negative/19778.jpg',
 'negative/19779.jpg',
 'negative/19780.jpg',
 'negative/19781.jpg',
 'negative/19782.jpg',
 'negative/19783.jpg',
 'negative/19784.jpg',
 'negative/19785.jpg',
 'negative/19786.jpg',
 'negative/19787.jpg',
 'negative/19788.jpg',
 'negative/19789.jpg',
 'negative/19790.jpg',
 'negative/19791.jpg',
 'negative/19792.jpg',
 'negative/19793.jpg',
 'negative/

In [0]:
import pandas as pd
results = pd.DataFrame({"Filename": filenames,"Probability_that_it_belongs_to_class 1":vgg16_predict[:,1]})

In [0]:
results

,Filename,Probability_that_it_belongs_to_class 1
0,negative/19751.jpg,0.987144
1,negative/19752.jpg,0.957533
2,negative/19753.jpg,0.971657
3,negative/19754.jpg,0.891699
4,negative/19755.jpg,0.988406
...,...,...
495,positive/19996.jpg,0.996941
496,positive/19997.jpg,0.869936
497,positive/19998.jpg,0.982298
498,positive/19999.jpg,0.083041


Author & Contributor: [Sandeep Aswathnarayana](https://www.linkedin.com/in/sandeep-a/)